In [ ]:
#pip install trino urllib3 pandas

In [ ]:
# sqlalchemy-trino has an integration bug w/ trino api, possibly fixed but unreleased
# also currently requires sqlalchemy 1.3, not 1.4
# however if they get this fixed it might be another useful connection api
#pip install --upgrade sqlalchemy==1.3 sqlalchemy-trino

In [ ]:
# This may not be compatible with installing sqlalchemy-trino
#pip install 'pyhive[presto,trino]'
# https://github.com/dropbox/PyHive

In [1]:
# two possible apis to generate a trino connection:
import trino
from pyhive import presto

# pandas dfs
import pandas as pd

import urllib3
urllib3.disable_warnings()

In [2]:
# In a live environment, it is best practice to fill these from environment variables
# that have been injected with proper values
trino_user = 'hukhan'
trino_passwd = 'osctrino987'
trino_host = 'trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org'
trino_port = 443

In [3]:
conn = trino.dbapi.connect(
    auth=trino.auth.BasicAuthentication(trino_user, trino_passwd),
    host=trino_host,
    port=trino_port,
    http_scheme='https',
    verify=False,
)
cur = conn.cursor()

In [4]:
cur.execute('show tables in hive.team1')
cur.fetchall()

[['cat']]

In [5]:
# this doesn't work with connection from trino api
# but it will work with connection from pyhive.presto, see below
#catDF = pd.read_sql("select * from hive.team1.cat", conn)

In [6]:
cur.execute('select * from hive.team1.cat')
df = pd.DataFrame(cur.fetchall())

# hack to move column names out of data
df, df.columns = df[1:], df.iloc[0]
df.reset_index(drop=True)

# infer data types (these will be str)
df = df.convert_dtypes()

# convert length to int
df['length'] = pd.to_numeric(df.length)

df.head()

,word,length
1,the,3
2,cat,3
3,in,2
4,the,3
5,hat,3


In [7]:
# check column types
df.dtypes

word      string
length     int64
dtype: object

## Using a pyhive.presto connection

In [8]:
import pyhive.presto
conn = presto.connect(
    username = trino_user,
    password = trino_passwd,
    host = trino_host,
    port = trino_port,
    protocol = 'https'
)

In [9]:
# With pyhive.presto you can load directly from the connection,
# although resulting DF is same as from trino api
df = pd.read_sql("select * from hive.team1.cat", conn)
df.head()

,word,length
0,word,length
1,the,3
2,cat,3
3,in,2
4,the,3


## **** Below is not working
see:
https://github.com/dungdm93/sqlalchemy-trino/issues/23

In [ ]:
sqlstring = 'trino://{user}:{passwd}@{host}:{port}/hive'.format(
    user = trino_user,
    passwd = trino_passwd,
    host=trino_host,
    port=trino_port
)
sqlstring

In [ ]:
import sqlalchemy
engine = sqlalchemy.engine.create_engine(
    sqlstring)

In [ ]:
df = pd.read_sql("select * from hive.team1.cat", engine)

In [ ]:
import sqlalchemy

sqlstring = 'trino://{user}:{passwd}@{host}:{port}/hive'.format( \
    user = trino_user, \
    passwd = trino_passwd, \
    host=trino_host, \
    port=trino_port
)
engine = sqlalchemy.create_engine(sqlstring)

In [ ]:
# https://github.com/dungdm93/sqlalchemy-trino/issues/23
pd.read_sql_query('select * from hive.team1.cat', sqlstring)